### Restart and Run All

In [2]:
import calendar
import os
import pandas as pd
import sidetable
import numpy as np
import panel as pn
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from itables import show
pn.extension('tabulator')

# Define a formatting function to format integers with zero decimal places and floats with two decimal places
def format_cell(x):
    if isinstance(x, int):
        return "{:,.0f}".format(x)
    elif isinstance(x, float):
        return "{:.2f}".format(x)
    else:
        return x

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

today = date.today()
print(today)

2025-02-23


### Tables in the process

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Weekly
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [7]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,WHART,10000,12.1,0,COS,SET,2024-11-25


In [8]:
names = orders['name']
name = names.to_string(index=False)

In [9]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHART,0.0756,0.1930,0.1930,0.1915,0.6531,"20,000",2025-03-03,2025-03-19,,1


In [10]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,149,WHART,0.00,0.00,S,8.80,12.10,0.00,0.00,12.30,"10,000",-4,4,"20,000",8.600000,11.740000,COS,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [12]:
sql = text('DELETE FROM orders')
rp = conlite.execute(sql)
rp.rowcount

20

In [13]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy.shape

(13, 7)

In [14]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell.shape

(10, 7)

In [15]:
#df_order = df_buy.append(df_sell, ignore_index=True)
df_order = pd.concat([df_buy, df_sell], ignore_index=True)
df_order.shape

(23, 7)

In [16]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,13,13,13,13,13,13
S,10,10,10,10,10,10


In [17]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(31, 2)

In [18]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(31, 9)

### If there is no dividend record, must create dividend record before proceed

In [20]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge


### Rerun from here

In [22]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,13,13,13,13,13,13,13
S,10,10,10,10,10,10,10


### Restart and Run All Cells

In [24]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,13,13,13,13,13,13,13
S,10,10,10,10,10,10,10


In [25]:
rcds = df_orders.values.tolist()
len(rcds)

23

In [26]:
for rcd in rcds:
    print(rcd)

['B', '3BBIF', 10000, 5.25, 0, 'RD50Pct', 'SET', '2025-02-18']
['B', 'AIMIRT', 5000, 10.2, 0, 'RND05pct', 'SET', '2024-11-26']
['S', 'AWC', 9000, 4.96, 0, 'COS', 'SET50', '2024-05-09']
['S', 'BCH', 2000, 21.7, 0, 'COS', 'SET100', '2024-08-29']
['B', 'CPNREIT', 5000, 10.8, 0, 'RD40Pct', 'SET', '2025-02-27']
['B', 'DIF', 5000, 8.3, 0, 'RD40Pct', 'SET', '2025-02-13']
['B', 'GVREIT', 5000, 6.2, 0, 'RND20Pct', 'SET', '2025-02-26']
['B', 'JMART', 3200, 11.2, 0, 'RND65pct', 'SET50', '2025-02-25']
['S', 'JMT', 900, 50.0, 0, 'COS', 'SET50', '2025-02-25']
['B', 'KCE', 1000, 22.8, 0, 'RD70Pct', 'SET100', '2024-08-26']
['B', 'MCS', 5000, 5.8, 0, 'RD60Pct', 'SET', '2025-04-18']
['B', 'NER', 6000, 4.7, 0, 'RD40pct', 'SET', '2025-04-24']
['S', 'PTG', 3600, 11.4, 0, 'COS', 'SET100', '2024-11-26']
['S', 'PTTGC', 1000, 30.0, 0, 'RD60pct', 'SET50', '2025-03-03']
['S', 'RCL', 3000, 38.75, 0, 'RD60pct', 'SET100', '2025-03-12']
['B', 'SENA', 7000, 1.92, 0, 'RD60pct', 'SET', '2024-05-10']
['B', 'STA', 2500, 

In [27]:
sql = text("""
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (:trade, :name, :qty, :price, :active, :reason, :market, :xdate)
""")

for rcd in rcds:
    # Convert list to dictionary with column names
    data = {
        'trade': rcd[0],
        'name': rcd[1],
        'qty': rcd[2],
        'price': rcd[3],
        'active': rcd[4],
        'reason': rcd[5],
        'market': rcd[6],
        'xdate': rcd[7]
    }
    
    conlite.execute(sql, data)
    conlite.commit()    

In [28]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders.shape

(23, 9)

In [29]:
file_name = 'orders.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [30]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\orders.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\orders.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\orders.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\orders.csv


In [31]:
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [34]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,8,B,JMART,3200,11.20,0,RND65pct,SET50,2025-02-25
1,3,S,AWC,9000,4.96,0,COS,SET50,2024-05-09
2,9,S,JMT,900,50.00,0,COS,SET50,2025-02-25
3,14,S,PTTGC,1000,30.00,0,RD60pct,SET50,2025-03-03


In [35]:
name = 'BLA'
limit = 20 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,20.000000,20.00000,20.000000,2.000000e+01,20.000000
mean,18.955000,19.26000,18.650000,1.322790e+06,18.945000
std,1.101423,1.17983,1.159174,1.440563e+06,1.132057
min,17.400000,17.80000,17.100000,2.236180e+05,17.400000
25%,18.075000,18.62500,17.775000,6.637630e+05,18.175000
50%,18.700000,19.10000,18.550000,8.669600e+05,18.850000
75%,19.400000,19.57500,19.075000,1.637924e+06,19.325000
max,21.500000,22.50000,21.400000,6.907848e+06,21.700000


### 2) Set100 records

In [37]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,10,B,KCE,1000,22.80,0,RD70Pct,SET100,2024-08-26
1,17,B,STA,2500,15.00,0,RD50Pct,SET100,2025-04-18
2,4,S,BCH,2000,21.70,0,COS,SET100,2024-08-29
3,13,S,PTG,3600,11.40,0,COS,SET100,2024-11-26
4,15,S,RCL,3000,38.75,0,RD60pct,SET100,2025-03-12


In [38]:
name = 'KKP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,52.066667,52.587500,51.454167,2.540200e+06,52.029167
std,1.321166,1.269613,1.106119,2.372426e+06,1.148038
min,49.750000,50.250000,49.000000,3.167920e+05,49.750000
25%,51.437500,51.937500,50.687500,1.423704e+06,51.250000
50%,52.000000,52.500000,51.500000,1.752802e+06,52.125000
75%,52.750000,53.000000,52.000000,2.763031e+06,52.750000
max,57.000000,57.500000,54.500000,1.485703e+07,55.250000


### 3) Set records

In [40]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,3BBIF,10000,5.25,0,RD50Pct,SET,2025-02-18
1,2,B,AIMIRT,5000,10.20,0,RND05pct,SET,2024-11-26
2,5,B,CPNREIT,5000,10.80,0,RD40Pct,SET,2025-02-27
3,6,B,DIF,5000,8.30,0,RD40Pct,SET,2025-02-13
4,7,B,GVREIT,5000,6.20,0,RND20Pct,SET,2025-02-26
5,11,B,MCS,5000,5.80,0,RD60Pct,SET,2025-04-18
6,12,B,NER,6000,4.70,0,RD40pct,SET,2025-04-24
7,16,B,SENA,7000,1.92,0,RD60pct,SET,2024-05-10
8,19,B,TFFIF,5000,6.00,0,RD20Pct,SET,2025-02-27
9,22,B,WHAIR,5000,6.00,0,RD30pct,SET,2025-03-03


In [41]:
name = 'SCCC'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
formatted_df = df.map(format_cell)

# Display the formatted dataframe
print(formatted_df)

     name        date   price    maxp    minp      qty    opnp
0    SCCC  2025-02-21  160.00  160.50  159.00   16,453  160.00
1    SCCC  2025-02-20  160.00  161.50  159.50   17,434  161.50
2    SCCC  2025-02-19  161.50  161.50  159.50   25,005  159.50
3    SCCC  2025-02-18  160.00  160.00  158.00  233,068  158.50
4    SCCC  2025-02-17  158.50  158.50  156.50   42,818  158.00
..    ...         ...     ...     ...     ...      ...     ...
115  SCCC  2023-09-05  138.00  138.00  137.50   14,733  138.00
116  SCCC  2023-09-04  138.00  138.00  137.00   24,099  137.00
117  SCCC  2023-09-01  137.00  138.50  137.00   52,618  138.50
118  SCCC  2023-08-31  138.50  139.00  138.00   61,042  139.00
119  SCCC  2023-08-30  138.50  139.50  138.50   20,726  139.50

[120 rows x 7 columns]


In [42]:
const.close()
conmy.close()
conlite.commit()
conlite.close()

In [43]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-23 20:33:35
